In [3]:
from pyspark.sql import SparkSession
from dotenv import load_dotenv
import os

In [4]:
# Cargar variables del entorno
load_dotenv("/home/jovyan/.env")

True

In [5]:
# Acceder a las credenciales
MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT")
MINIO_ACCESS_KEY = os.getenv("MINIO_ROOT_USER")
MINIO_SECRET_KEY = os.getenv("MINIO_ROOT_PASSWORD")
BUCKET = os.getenv("MINIO_BUCKET_BRONZE")  # Podés probar con SILVER si querés

In [6]:
print(BUCKET)

dev-bronze


In [7]:
# Crear sesión Spark con soporte para s3a
spark = SparkSession.builder \
    .appName("Test MinIO") \
    .config("spark.jars", "/usr/local/spark/jars/hadoop-aws-3.3.4.jar,/usr/local/spark/jars/aws-java-sdk-bundle-1.11.901.jar") \
    .config("spark.hadoop.fs.s3a.endpoint", MINIO_ENDPOINT) \
    .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET_KEY) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

25/05/15 14:46:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [8]:
# Crear un DataFrame de prueba
df = spark.createDataFrame([
    ("Lisandro", 3),
    ("Paola", 34),
    ("Felo", 24)
], ["nombre", "edad"])

In [9]:
df.show()

+--------+----+
|  nombre|edad|
+--------+----+
|Lisandro|   3|
|   Paola|  34|
|    Felo|  24|
+--------+----+



In [10]:
# Guardar como Parquet en MinIO (bucket BRONZE)
output_path = f"s3a://{BUCKET}/test_spark_parquet"

In [11]:
print(output_path)

s3a://dev-bronze/test_spark_parquet


In [12]:
df = df.repartition(1)

In [13]:
df.write.mode("overwrite").parquet(output_path)

print(f"✅ Archivo guardado en {output_path}")

25/05/15 14:46:53 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


✅ Archivo guardado en s3a://dev-bronze/test_spark_parquet
